# Content-based recommender system

Bibliotecas necessarias

In [120]:
import pandas as pd
import numpy as np

Dados dos filmes e generos

In [53]:
movies_df = pd.read_csv('movies.csv')
movies_df.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Notas para filmes

In [28]:
ratings_df = pd.read_csv('ratings.csv')
ratings_df.head(10)

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496
5,2,112552,5.0,1436165496
6,2,112556,4.0,1436165499
7,3,356,4.0,920587155
8,3,2394,4.0,920586920
9,3,2431,5.0,920586945


Gerando feature com o ano do filme a partir do titulo

In [54]:
#creating movie year feature
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))', expand=False)
#removing parenthesis
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)', expand=False)
#removing the year from the title
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df.title.apply(lambda x: x.strip())
movies_df

C:\Users\R2\AppData\Local\Temp\ipykernel_7092\4280451533.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
34203,151697,Grand Slam,Thriller,1967
34204,151701,Bloodmoney,(no genres listed),2010
34205,151703,The Butterfly Circus,Drama,2009
34206,151709,Zero,Drama|Sci-Fi,2015


Separando os generos dos filmes em uma lista

In [55]:
movies_df.genres = movies_df.genres.str.split('|')
movies_df

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995
...,...,...,...,...
34203,151697,Grand Slam,[Thriller],1967
34204,151701,Bloodmoney,[(no genres listed)],2010
34205,151703,The Butterfly Circus,[Drama],2009
34206,151709,Zero,"[Drama, Sci-Fi]",2015


Aplicando um one hot encoder com os generos dos filmes

In [56]:
movies_genres_df = movies_df.copy()

for index, row in movies_df.iterrows():
    for genre in row['genres']:
        movies_genres_df.at[index, genre] = 1

movies_genres_df = movies_genres_df.fillna(0)
movies_genres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Analisando os dados com as notas dos filmes

In [57]:
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [58]:
ratings_df.drop(columns=['timestamp'], inplace=True)

KeyError: "['timestamp'] not found in axis"

Criando a simulação de um perfil com notas de alguns filmes

In [59]:
movies_df.query('title.str.contains("Harry Potter")', engine='python').title

4801     Harry Potter and the Sorcerer's Stone (a.k.a. ...
5718               Harry Potter and the Chamber of Secrets
7770              Harry Potter and the Prisoner of Azkaban
10610                  Harry Potter and the Goblet of Fire
11986            Harry Potter and the Order of the Phoenix
13957               Harry Potter and the Half-Blood Prince
16222         Harry Potter and the Deathly Hallows: Part 1
17537         Harry Potter and the Deathly Hallows: Part 2
Name: title, dtype: object

In [60]:
movies_df.query('title.str.contains("Fast and")', engine='python').title

4275                             Fast and the Furious, The
6278        2 Fast 2 Furious (Fast and the Furious 2, The)
11111    Fast and the Furious: Tokyo Drift, The (Fast a...
13644         Fast & Furious (Fast and the Furious 4, The)
13976                            Fast and the Furious, The
17152              Fast Five (Fast and the Furious 5, The)
19624                             Hard, Fast and Beautiful
21113       Fast & Furious 6 (Fast and the Furious 6, The)
30309                Tom and Jerry: The Fast and the Furry
Name: title, dtype: object

In [61]:
movies_df.query('title.str.contains("Twilight Saga")', engine='python').title

14490                  Twilight Saga: New Moon, The
15489                   Twilight Saga: Eclipse, The
18251    Twilight Saga: Breaking Dawn - Part 1, The
19951    Twilight Saga: Breaking Dawn - Part 2, The
Name: title, dtype: object

Supondo que o perfil goste de harry potter, goste mais ou menos de velozes e furiosos e não goste da saga crepusculo

In [74]:
#ratings from a user 
userInput = [
            {'title':'Harry Potter and the Deathly Hallows: Part 1', 'rating':4.8},
            {'title':'Harry Potter and the Deathly Hallows: Part 2', 'rating':5},
            {'title':'Twilight Saga: New Moon, The', 'rating':2.3},
            {'title':'Twilight Saga: Eclipse, The', 'rating':2.5},
            {'title':"Fast & Furious 6 (Fast and the Furious 6, The)", 'rating':3.5},
            {'title':'2 Fast 2 Furious (Fast and the Furious 2, The)', 'rating':4}
         ] 

inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Harry Potter and the Deathly Hallows: Part 1,4.8
1,Harry Potter and the Deathly Hallows: Part 2,5.0
2,"Twilight Saga: New Moon, The",2.3
3,"Twilight Saga: Eclipse, The",2.5
4,"Fast & Furious 6 (Fast and the Furious 6, The)",3.5
5,"2 Fast 2 Furious (Fast and the Furious 2, The)",4.0


In [75]:
perfil_movies_title = inputMovies.title.tolist()
perfil_movies_title

['Harry Potter and the Deathly Hallows: Part 1',
 'Harry Potter and the Deathly Hallows: Part 2',
 'Twilight Saga: New Moon, The',
 'Twilight Saga: Eclipse, The',
 'Fast & Furious 6 (Fast and the Furious 6, The)',
 '2 Fast 2 Furious (Fast and the Furious 2, The)']

Verificando os filmes do usuario no dataset referente aos filmes

In [76]:
inputId = movies_df[movies_df['title'].isin(perfil_movies_title)]
inputId

,movieId,title,genres,year
6278,6383,"2 Fast 2 Furious (Fast and the Furious 2, The)","[Action, Crime, Thriller]",2003
14490,72407,"Twilight Saga: New Moon, The","[Drama, Fantasy, Horror, Romance, Thriller]",2009
15489,78772,"Twilight Saga: Eclipse, The","[Fantasy, Romance, Thriller, IMAX]",2010
16222,81834,Harry Potter and the Deathly Hallows: Part 1,"[Action, Adventure, Fantasy, IMAX]",2010
17537,88125,Harry Potter and the Deathly Hallows: Part 2,"[Action, Adventure, Drama, Fantasy, Mystery, I...",2011
21113,102716,"Fast & Furious 6 (Fast and the Furious 6, The)","[Action, Crime, Thriller, IMAX]",2013


Realizando merge para apresentar tanto o movieId quanto a nota do usuario

In [77]:
inputMovies = pd.merge(inputId, inputMovies)
inputMovies

,movieId,title,genres,year,rating
0,6383,"2 Fast 2 Furious (Fast and the Furious 2, The)","[Action, Crime, Thriller]",2003,4.0
1,72407,"Twilight Saga: New Moon, The","[Drama, Fantasy, Horror, Romance, Thriller]",2009,2.3
2,78772,"Twilight Saga: Eclipse, The","[Fantasy, Romance, Thriller, IMAX]",2010,2.5
3,81834,Harry Potter and the Deathly Hallows: Part 1,"[Action, Adventure, Fantasy, IMAX]",2010,4.8
4,88125,Harry Potter and the Deathly Hallows: Part 2,"[Action, Adventure, Drama, Fantasy, Mystery, I...",2011,5.0
5,102716,"Fast & Furious 6 (Fast and the Furious 6, The)","[Action, Crime, Thriller, IMAX]",2013,3.5


In [78]:
inputMovies = inputMovies.drop(columns=['genres','year'])

Id dos filmes do usuario

In [79]:
perfil_movies_id = inputMovies.movieId.tolist()
perfil_movies_id

[6383, 72407, 78772, 81834, 88125, 102716]

Buscando os filmes pelo ID na tabela com os generos codificados

In [80]:
userMovies = movies_genres_df[movies_genres_df['movieId'].isin(perfil_movies_id)]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
6278,6383,"2 Fast 2 Furious (Fast and the Furious 2, The)","[Action, Crime, Thriller]",2003,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14490,72407,"Twilight Saga: New Moon, The","[Drama, Fantasy, Horror, Romance, Thriller]",2009,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15489,78772,"Twilight Saga: Eclipse, The","[Fantasy, Romance, Thriller, IMAX]",2010,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
16222,81834,Harry Potter and the Deathly Hallows: Part 1,"[Action, Adventure, Fantasy, IMAX]",2010,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
17537,88125,Harry Potter and the Deathly Hallows: Part 2,"[Action, Adventure, Drama, Fantasy, Mystery, I...",2011,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
21113,102716,"Fast & Furious 6 (Fast and the Furious 6, The)","[Action, Crime, Thriller, IMAX]",2013,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Para quantificar o peso dos generos dos filmes, deixar o dataset apenas com as colunas referentes aos generos;

In [81]:
userMovies = userMovies.reset_index(drop=True)
userGenreTable = userMovies.drop(columns=['movieId','title','genres','year'])

In [82]:
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Avaliando a ordem de preferência dos genêros  assistidos pelo perfil

In [103]:
userProfile = userGenreTable.T.dot(inputMovies['rating'])
userProfile.sort_values(ascending=False)

Action                17.3
IMAX                  15.8
Fantasy               14.6
Thriller              12.3
Adventure              9.8
Crime                  7.5
Drama                  7.3
Mystery                5.0
Romance                4.8
Horror                 2.3
Animation              0.0
Comedy                 0.0
Sci-Fi                 0.0
Children               0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [112]:
genreTable = movies_genres_df.set_index(movies_genres_df['movieId'])
genreTable.drop(columns=['movieId','title','genres','year'], inplace=True)
genreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
151703,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
genreTable.shape

(34208, 20)

Peso de cada filme de acordo com as preferências do usuário

In [106]:
recommendation_table = ((genreTable*userProfile)).sum(axis=1)/(userProfile.sum())
recommendation_table = recommendation_table.sort_values(ascending=False)
recommendation_table.head(20)

movieId
88125     0.721820
115669    0.721820
83349     0.698035
71999     0.683557
79132     0.674250
86332     0.670114
84613     0.646329
91535     0.646329
78105     0.644261
81132     0.635988
41569     0.633919
2429      0.633919
117646    0.633919
86644     0.622544
85510     0.620476
59844     0.612203
67070     0.612203
459       0.610134
2414      0.610134
43932     0.608066
dtype: float64

Recomendacao de 20 filmes pro usuario

In [109]:
movies_df.loc[movies_df['movieId'].isin(recommendation_table.head(20).keys())]

,movieId,title,genres,year
455,459,"Getaway, The","[Action, Adventure, Crime, Drama, Romance, Thr...",1994
2330,2414,Young Sherlock Holmes,"[Action, Adventure, Children, Fantasy, Mystery...",1985
2345,2429,Mighty Joe Young,"[Action, Adventure, Drama, Fantasy, Thriller]",1998
10654,41569,King Kong,"[Action, Adventure, Drama, Fantasy, Thriller]",2005
10872,43932,Pulse,"[Action, Drama, Fantasy, Horror, Mystery, Sci-...",2006
12720,59844,"Honor Among Thieves (Adieu l'ami) (Farewell, F...","[Action, Adventure, Crime, Drama, Mystery, Thr...",1968
13552,67070,Army of One (Joshua Tree),"[Action, Adventure, Crime, Drama, Mystery, Thr...",1993
14397,71999,Aelita: The Queen of Mars (Aelita),"[Action, Adventure, Drama, Fantasy, Romance, S...",1924
15346,78105,Prince of Persia: The Sands of Time,"[Action, Adventure, Fantasy, Romance, IMAX]",2010
15562,79132,Inception,"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill...",2010


# Collaborative Filtering recommender system

In [121]:
import pandas as pd
import numpy as np

lendo dataset com os filmes e generos

In [134]:
movies_df = pd.read_csv('movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [131]:
movies_df.shape

(34208, 3)

Lendo dataset com notas de usuários para os filmes

In [123]:
ratings_df = pd.read_csv('ratings.csv')
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [127]:
ratings_df.drop(columns=['timestamp'], inplace=True)

In [125]:
ratings_df.shape

(22884377, 4)

Tratando os dados dos filmes

In [135]:
#criando a variavel com o ano do filme a partir do titulo
movies_df['year'] = movies_df['title'].str.extract('(\(\d\d\d\d\))', expand=False)
movies_df['year'] = movies_df['year'].str.extract('(\d\d\d\d)', expand=False)
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [136]:
#retirando o ano do titulo
movies_df.title = movies_df.title.str.replace('(\(\d\d\d\d\))', '')

#retirando possiveis espaços no final do titulo
movies_df.title = movies_df.title.apply(lambda x: x.strip())

C:\Users\R2\AppData\Local\Temp\ipykernel_7092\3522699013.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df.title = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


In [138]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [140]:
#para esse tipo de sistema de recomendação, os generos dos filmes não possuem importancia

movies_df.drop(columns=['genres'], inplace=True)

Supondo que o perfil goste de harry potter, goste mais ou menos de velozes e furiosos e não goste da saga crepusculo

In [141]:
#ratings from a user 
userInput = [
            {'title':'Harry Potter and the Deathly Hallows: Part 1', 'rating':4.8},
            {'title':'Harry Potter and the Deathly Hallows: Part 2', 'rating':5},
            {'title':'Twilight Saga: New Moon, The', 'rating':2.3},
            {'title':'Twilight Saga: Eclipse, The', 'rating':2.5},
            {'title':"Fast & Furious 6 (Fast and the Furious 6, The)", 'rating':3.5},
            {'title':'2 Fast 2 Furious (Fast and the Furious 2, The)', 'rating':4}
         ] 

inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Harry Potter and the Deathly Hallows: Part 1,4.8
1,Harry Potter and the Deathly Hallows: Part 2,5.0
2,"Twilight Saga: New Moon, The",2.3
3,"Twilight Saga: Eclipse, The",2.5
4,"Fast & Furious 6 (Fast and the Furious 6, The)",3.5
5,"2 Fast 2 Furious (Fast and the Furious 2, The)",4.0


In [144]:
#encontrando o id dos filmes do usuario no dataset original
inputId = movies_df[movies_df.title.isin(inputMovies.title)]
inputId

,movieId,title,year
6278,6383,"2 Fast 2 Furious (Fast and the Furious 2, The)",2003
14490,72407,"Twilight Saga: New Moon, The",2009
15489,78772,"Twilight Saga: Eclipse, The",2010
16222,81834,Harry Potter and the Deathly Hallows: Part 1,2010
17537,88125,Harry Potter and the Deathly Hallows: Part 2,2011
21113,102716,"Fast & Furious 6 (Fast and the Furious 6, The)",2013


In [145]:
#realizando merge para apresentar tanto o movieId quanto o rating
inputMovies = pd.merge(inputId, inputMovies)
inputMovies

,movieId,title,year,rating
0,6383,"2 Fast 2 Furious (Fast and the Furious 2, The)",2003,4.0
1,72407,"Twilight Saga: New Moon, The",2009,2.3
2,78772,"Twilight Saga: Eclipse, The",2010,2.5
3,81834,Harry Potter and the Deathly Hallows: Part 1,2010,4.8
4,88125,Harry Potter and the Deathly Hallows: Part 2,2011,5.0
5,102716,"Fast & Furious 6 (Fast and the Furious 6, The)",2013,3.5


Procurando os usuarios que assistiram os mesmos filmes

In [146]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies.movieId)]
userSubset

,userId,movieId,rating
7472,71,81834,5.0
8008,78,6383,3.5
11141,114,81834,2.0
11585,120,6383,1.5
11732,120,72407,2.0
...,...,...,...
22879454,247700,6383,2.0
22879701,247700,81834,4.0
22879713,247700,88125,3.0
22882422,247725,6383,2.0


Agrupando as avaliações por usuario

In [147]:
userSubsetGroup = userSubset.groupby('userId')

In [151]:
#analisando o resultado de algum usuario qualquer
userSubsetGroup.get_group(247700)

,userId,movieId,rating
22879454,247700,6383,2.0
22879701,247700,81834,4.0
22879713,247700,88125,3.0


analisando quais usuarios assistiram a maior quantidade de filmes que o perfil assistiu

In [163]:
userSubsetGroup = sorted(userSubsetGroup, key= lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]

[(1573,
          userId  movieId  rating
  140135    1573     6383     4.0
  140590    1573    72407     3.5
  140633    1573    78772     3.0
  140668    1573    81834     5.0
  140724    1573    88125     5.0
  140904    1573   102716     4.0),
 (6530,
          userId  movieId  rating
  613163    6530     6383     2.5
  613885    6530    72407     2.5
  613928    6530    78772     3.0
  613960    6530    81834     4.0
  614011    6530    88125     3.5
  614134    6530   102716     2.5),
 (9431,
          userId  movieId  rating
  868594    9431     6383     4.0
  868723    9431    72407     2.5
  868736    9431    78772     2.0
  868747    9431    81834     4.0
  868762    9431    88125     3.5
  868813    9431   102716     4.0)]

#### Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.\
We're going to find out how similar each user is to the input through the **Pearson Correlation Coefficient**. It is used to measure the strength of a linear association between the two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y, then, pearson(X, Y) == pearson(X, 2 \* Y + 3). This is a pretty important property in recommendation systems because, for example, two users might rate two series of items totally differently in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation.

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.


In [195]:
userSubsetGroup100 = userSubsetGroup[0:100]

In [196]:
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/np.sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [200]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.948681,1573
1,0.669340,6530
2,0.762144,9431
3,-0.525569,11905
4,0.986032,15474


In [202]:
topUsers = pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
4656,1.0,180310
2164,1.0,33165
3641,1.0,120540
5701,1.0,247285
5378,1.0,225334


In [203]:
topUsersRating = topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,180310,1,3.0
1,1.0,180310,2,3.5
2,1.0,180310,32,4.5
3,1.0,180310,47,4.5
4,1.0,180310,50,4.5


In [204]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,180310,1,3.0,3.0
1,1.0,180310,2,3.5,3.5
2,1.0,180310,32,4.5,4.5
3,1.0,180310,47,4.5,4.5
4,1.0,180310,50,4.5,4.5


In [205]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,28.0,108.5
2,15.0,49.5
3,5.0,16.5
5,8.0,20.5
6,7.0,28.0


In [206]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.8750,1
2,3.3000,2
3,3.3000,3
5,2.5625,5
6,4.0000,6


In [207]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
56012,5.0,56012
326,5.0,326
1941,5.0,1941
2840,5.0,2840
5649,5.0,5649
952,5.0,952
111592,5.0,111592
99349,5.0,99349
111449,5.0,111449


In [208]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
322,326,To Live (Huozhe),1994
935,952,Around the World in 80 Days,1956
1858,1941,Hamlet,1948
2755,2840,Stigmata,1999
5551,5649,Horror of Dracula (Dracula),1958
8411,25827,Mr. Deeds Goes to Town,1936
12252,56012,"Evening with Kevin Smith 2: Evening Harder, An",2006
20241,99349,Kevin Smith: Burn in Hell,2012
23543,111449,Burton and Taylor,2013
23574,111592,Birth of the Living Dead,2013
